In [1]:
import glob
import numpy as np
import cv2
from PIL import Image #testando
import imutils #testando
#from collections import namedtuple #testando
from skimage import filters
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (11,7) #apresenta todas as imagens na proporção definida

In [2]:
def crop_name_seg(img):
    imgcrop = img[100:100+40, 90:90+490] 
    return imgcrop

In [3]:
def cannyedges(img):
    edges = cv2.Canny(img,80,150)
    return edges

In [4]:
def dilatacao(img):
    kernel = np.ones((2,2),np.uint8)
    dilation = cv2.dilate(img,kernel,iterations = 1)
    return dilation

In [5]:
def erosao(img):
    kernel = np.ones((2,2),np.uint8)
    erosion = cv2.erode(img,kernel)
    return erosion

In [6]:
def opening(img):
    kernel = np.ones((2,2),np.uint8)
    op = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    return op

In [7]:
def gaussianBlurring(img):
    blur1 = cv2.GaussianBlur(img, (5, 5), 0)
    return blur1

In [8]:
def conv_edges(img):
    convolution_matrix = np.array([[0,1,0],
                                  [1,-4,1],
                                  [0,1,0]]) #filtro/matriz para detecção de bordas
    edg = cv2.filter2D(img, -1, convolution_matrix)
    return edg

In [9]:
def sharpenFunction(image):
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]) #matriz de convolução que aplica nitidez
    return cv2.filter2D(image, -1, kernel)

In [10]:
def inverte(img):
    ret,thresh1 = cv2.threshold(img,87,255,cv2.THRESH_BINARY)
    #ret,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
    return thresh1

In [11]:
def settozero(img):
    ret, thresh = cv2.threshold(img, 170, 255, cv2.THRESH_TOZERO)
    return thresh

In [12]:
def thresholding(image):
    (T,Thresh1) = cv2.threshold(image, 185, 255, cv2.THRESH_TRUNC)#Valores maiores do que o limite são ajustados iguais ao valor do limite e os valores menores do que o limite permanecem inalterados.
    (T,Thresh3) = cv2.threshold(Thresh1, 43, 255, cv2.THRESH_BINARY)#transforma qualquer valor acima do limite em valor mínimo possível e qualquer valor abaixo do limite em valor máximo possível.
    (T,Thresh2) = cv2.threshold(Thresh3, 0 ,255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C)#o valor limite no pixel (x, y) é a soma ponderada gaussiana dos valores de vizinhança menos alguma constante C.
    (T,Thresh4) = cv2.threshold(Thresh2, 30, 255, cv2.CALIB_CB_ADAPTIVE_THRESH)
    return Thresh3

In [13]:
def togray(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return g

In [14]:
def filterFunction(image):
    (T,Thresh1) = cv2.threshold(image, 200, 255, cv2.THRESH_TRUNC)
    Thresh2 = cv2.adaptiveThreshold(Thresh1, 255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 231,50)
    return Thresh2

In [15]:
def bilateral(img):
    bil = cv2.bilateralFilter(img, 11, 77, 77) #3, 21, 21 #5, 35, 35 #7, 49, 49 #9, 63, 63 #11, 77, 77
    return bil

In [16]:
def thresholdLocal(img):
    T = skimage.filters.threshold_local(img, 11, offset = 10, method = "gaussian")
    imag = (gray > T).astype("uint8") * 255
    return imag

In [17]:
def cleanImage(image, stage = 0):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    # aplicando as operações topHat/blackHat
    topHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, kernel) #revela as áreas claras com fundo escuro
    blackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel) #revela as áreas escuras com um fundo claro
    # fazendo adição e subtração entre as operações morfológicas
    add = cv2.add(gray, topHat)
    subtract = cv2.subtract(add, blackHat)
    if (stage == 1):
        return subtract
    T = filters.threshold_local(subtract, 29, offset=7, method="gaussian", mode="mirror")
    thresh = (subtract > T).astype("uint8") * 255
    if (stage == 2):
        return thresh
    # inverte a imagem 
    thresh = cv2.bitwise_not(thresh)
    return thresh

In [18]:
# images = glob.glob("../alinhadas/*jpg")

# for j in images:
#     image = cv2.imread(j)